In [ ]:
pip install thefuzz

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from thefuzz import process  # Fuzzy matching 라이브러리

# CSV 파일 로드
df = pd.read_csv('preprocessed_비타민C.csv')

# 결측값 제거 (기능성 컬럼에 대해서만 처리)
df = df.dropna(subset=['기능성'])

# 데이터 확인 (기능성 컬럼에 어떤 값들이 있는지 확인)
print("기능성 컬럼의 값들:", df['기능성'].unique())  # 기능성 컬럼 값 확인

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['기능성'])

# **📌 키워드를 TF-IDF로 검색하는 방식으로 변경**
def recommend(keyword, limit=5, fuzzy_threshold=70):  # fuzzy_threshold 낮추기
    """키워드 기반 제품 추천 (Fuzzy Matching 포함)"""
    
    # 🔹 키워드를 Fuzzy Matching으로 확장 (유사어 찾기)
    expanded_keywords = get_fuzzy_matches(keyword, fuzzy_threshold)
    print(f"확장된 키워드들: {expanded_keywords}")  # 확장된 키워드 출력
    
    if not expanded_keywords:
        return "유사어가 없거나 기준 유사도에 맞는 항목이 없습니다."
    
    # 🔹 확장된 키워드로 추천
    recommended_products = pd.DataFrame(columns=['제품명', '유사도'])
    
    for word in expanded_keywords:
        # 🔹 키워드를 TF-IDF로 변환
        keyword_vec = tfidf.transform([word])

        # 🔹 키워드와 모든 제품 간 코사인 유사도 계산
        sim_scores = cosine_similarity(keyword_vec, tfidf_matrix)[0]

        # 🔹 유사도가 높은 상위 limit개 제품 가져오기
        top_indices = sim_scores.argsort()[-limit:][::-1]

        for idx in top_indices:
            recommended_products = recommended_products.append({'제품명': df['제품명'].iloc[idx], '유사도': sim_scores[idx]}, ignore_index=True)
    
    # 🔹 유사도 기준으로 정렬
    recommended_products = recommended_products.sort_values(by='유사도', ascending=False).drop_duplicates('제품명')
    
    return recommended_products[['제품명', '유사도']].head(limit)

def get_fuzzy_matches(query, threshold=70):
    """ Fuzzy Matching을 통해 유사어를 찾는 함수 """
    # 유사어 후보 리스트 (기존 제품명, 성분, 기능성 등을 이용)
    candidate_words = df['기능성'].unique().tolist()
    
    # 후보 리스트 출력 (디버깅용)
    print(f"Fuzzy Matching 후보 리스트: {candidate_words[:20]}")  # 처음 20개만 출력

    # Fuzzy Matching으로 유사어 추출
    matches = process.extract(query, candidate_words, limit=10)
    print(f"매칭 결과: {matches}")  # Fuzzy Matching 결과 출력
    return [match[0] for match in matches if match[1] >= threshold]

# 예시 테스트: '비타민'과 관련된 추천 제품
recommend_results = recommend('비타민c', limit=5)
print(recommend_results)


기능성 컬럼의 값들: ['[회화나무열매추출물] 갱년기 여성의 건강에 도움을 줄 수 있음(기타기능Ⅱ)\n\n[비타민C] ①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요\n\n[비타민B1] ①탄수화물과 에너지 대사에 필요\n\n[나이아신] ①체내 에너지 생성에 필요\n\n[비타민B6] ①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n[비타민B2] ①체내 에너지 생성에 필요\n\n[판토텐산] ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[비오틴] ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[엽산] ①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n[비타민B12] ①정상적인 엽산 대사에 필요\n\n[비타민D] ①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌\n\n[비타민E] ①유해산소로부터 세포를 보호하는데 필요\n\n[아연] ①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n[망간] ①뼈 형성에 필요②에너지 이용에 필요③유해산소로부터 세포를 보호하는데 필요\n\n[마그네슘] ①에너지 이용에 필요②신경과 근육 기능 유지에 필요\n\n[칼슘] ①뼈와 치아 형성에 필요②신경과 근육 기능 유지에 필요③정상적인 혈액응고에 필요④골다공증발생 위험 감소에 도움을 줌\n\n[셀레늄(또는 셀렌)] ①유해산소로부터 세포를 보호하는데 필요\n\n[구리] ①철의 운반과 이용에 필요②유해산소로부터 세포를 보호하는데 필요\n\n[베타카로틴] ①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요'
 '[저분자 피쉬 콜라겐 펩타이드 (Naticol® BPMG)(제2023-25호)](국문)① 피부 보습에 도움을 줄 수 있음 ② 자외선에 의한 피부 손상으로부터 피부 건강을  유지하는데 도움을 줄 수 있